In [1]:
from graphlib.structures import Graph
from graphlib.algorithms import DFS_with_cc, BFS_search
from graphlib.tools import radius_approximate, diameter_approximate

In [2]:
%%time

G = Graph('CO-Astro')
with open('./datasets/CA-AstroPh.txt', mode='r') as f:
    for line in f:
        u, v = line.split()
        G.add_edge(int(u), int(v))
print(G)

Graph <CO-Astro> with 18772 nodes and 198110 edges
Wall time: 544 ms


In [3]:
%%time

number, largest_index, components = DFS_with_cc(G, largest=True)
print(f'Наибольшая компонента состоит из {len(components[largest_index])} вершин')
largest_component = G.subgraph(nodes=components[largest_index])
print(largest_component)

Наибольшая компонента состоит из 17903 вершин
Graph <subgraph::CO-Astro> with 17903 nodes and 197031 edges
Wall time: 127 ms


In [4]:
#nodes_for_evaluate = largest_component.random_selection(x=70)
nodes_for_evaluate = [68025, 747, 131215, 126491, 34890, 109268, 51594, 116605, 119616, 79297, 8953, 49130, 15352, 64679, 63022, 296, 63171, 28239, 36608, 27175, 51015, 112868, 94726, 34085, 16303, 31345, 122817, 118173, 73207, 31878, 80496, 24079, 22528, 44467, 44623, 124056, 119434, 57988, 12504, 107023, 26325, 108289, 30007, 66518, 14804, 108207, 105621, 25533, 115965, 83835, 106340, 18444, 4971, 79413, 114790, 88349, 128370, 27147, 64860, 122431, 121891, 2417, 93689, 22834, 19395, 124682, 67090, 32051, 94817, 23893]
print(nodes_for_evaluate)

[68025, 747, 131215, 126491, 34890, 109268, 51594, 116605, 119616, 79297, 8953, 49130, 15352, 64679, 63022, 296, 63171, 28239, 36608, 27175, 51015, 112868, 94726, 34085, 16303, 31345, 122817, 118173, 73207, 31878, 80496, 24079, 22528, 44467, 44623, 124056, 119434, 57988, 12504, 107023, 26325, 108289, 30007, 66518, 14804, 108207, 105621, 25533, 115965, 83835, 106340, 18444, 4971, 79413, 114790, 88349, 128370, 27147, 64860, 122431, 121891, 2417, 93689, 22834, 19395, 124682, 67090, 32051, 94817, 23893]


In [5]:
diameter_approximate(largest_component, nodes=nodes_for_evaluate, with_nodes=True)

(8, 31345, 79413)

In [6]:
radius_approximate(largest_component, nodes=nodes_for_evaluate, with_nodes=True)

(3, 94817, 23893)